In [1]:
import os
import pandas as pd
import re
import matplotlib.pyplot as plt

In [2]:
# Get current directory and change to the data stored path
print("Current directory:",os.getcwd())

path = "/stornext/HPCScratch/rcp_sf_data/starfish_data/summary_by_day"
os.chdir(path)

# List all files 
all_files = os.listdir(path)

Current directory: /stornext/Home/data/allstaff/z/zhou.yo


In [3]:
# Concat all files into one file
file_list = []
for filename in all_files:
    df = pd.read_csv(filename)
    df['filename'] = filename
    file_list.append(df)
data = pd.concat(file_list, axis=0, ignore_index=True)

In [4]:
data.dtypes

volume name                              object
capacity (GiB)                          float64
used physical (GiB)                     float64
used logical (GiB)                      float64
free (GiB)                              float64
used %                                  float64
regular files                            object
symbolic links                           object
max file size (GiB)                     float64
median file size (bytes)                 object
average file size (bytes)                object
directories                              object
directories with subdirectories only     object
empty directories                        object
max dir size (GiB)                      float64
run_time                                 object
filename                                 object
dtype: object

## Clean Volume name

In [5]:
pattern = "[A-Z]\w*"
cleaned_name = [re.search(pattern,volume_name).group() for volume_name in data["volume name"]]
data["volume name"] = cleaned_name

In [6]:
data.head()

,volume name,capacity (GiB),used physical (GiB),used logical (GiB),free (GiB),used %,regular files,symbolic links,max file size (GiB),median file size (bytes),average file size (bytes),directories,directories with subdirectories only,empty directories,max dir size (GiB),run_time,filename
0,Bioinf,476836.88,452991.40,2045229.43,23845.47,95.0,65081232,696836,7687.31,5598,33780535,3467941,923935,422101,46895.55,18/11/20 4:00,2020-11-18.csv
1,Projects,366941.01,159508.05,264816.49,207432.96,43.5,43451260,153389,635.62,48457,6543863,445345,177355,124779,7751.23,18/11/20 4:00,2020-11-18.csv
2,General,500562.34,329524.32,1099954.85,171038.02,65.8,237069959,2182642,954.28,46818,4984182,6698668,1265674,634557,8453.05,18/11/20 4:00,2020-11-18.csv
3,Genomics,62570.29,41686.21,88837.25,20884.08,66.6,5306768,93224,188.85,26898,17974782,1225326,318485,278915,3305.27,18/11/20 4:00,2020-11-18.csv
4,Home,238418.44,93698.70,507026.26,144719.74,39.3,117911464,1080222,623.61,6585,4619693,8589748,2078735,1101354,3065.48,18/11/20 4:00,2020-11-18.csv


## Clean Date Format

In [7]:
date = [time.split()[0] for time in data["run_time"]]

In [8]:
date = pd.to_datetime(date,format='%d/%m/%y') 
data["date"] = date

In [9]:
data.dtypes

volume name                                     object
capacity (GiB)                                 float64
used physical (GiB)                            float64
used logical (GiB)                             float64
free (GiB)                                     float64
used %                                         float64
regular files                                   object
symbolic links                                  object
max file size (GiB)                            float64
median file size (bytes)                        object
average file size (bytes)                       object
directories                                     object
directories with subdirectories only            object
empty directories                               object
max dir size (GiB)                             float64
run_time                                        object
filename                                        object
date                                    datetime64[ns]
dtype: obj

## Keep the physical and logical columns

In [10]:
data = data[['date','volume name', 'capacity (GiB)', 'used physical (GiB)',
       'used logical (GiB)', 'free (GiB)', 'used %']]

## Missing and Noisy data clean


In [11]:
data = data.sort_values(["volume name", "date"], ascending = (True, True))
data = data.reset_index(drop = True)

In [12]:
data

,date,volume name,capacity (GiB),used physical (GiB),used logical (GiB),free (GiB),used %
0,2020-11-03,Admin,32767.98,1080.94,996.14,31687.05,3.3
1,2020-11-04,Admin,32767.98,1080.94,996.14,31687.05,3.3
2,2020-11-05,Admin,32767.98,1080.94,996.14,31687.04,3.3
3,2020-11-06,Admin,32767.98,1080.94,996.14,31687.04,3.3
4,2020-11-07,Admin,32767.98,1080.94,996.14,31687.04,3.3
...,...,...,...,...,...,...,...
2816,2021-06-06,System,32767.98,9943.52,87830.62,22824.47,30.3
2817,2021-06-07,System,32767.98,10135.55,87990.45,22632.43,30.9
2818,2021-06-08,System,32767.98,10327.59,88179.21,22440.39,31.5
2819,2021-06-09,System,32767.98,10567.96,88410.56,22200.02,32.3


In [13]:
# There are some noisy total capacity values with number  9.99
data[data["capacity (GiB)"] < 100]

,date,volume name,capacity (GiB),used physical (GiB),used logical (GiB),free (GiB),used %
42,2020-12-15,Admin,9.99,1081.07,996.27,0.0,100.0
162,2021-04-14,Admin,9.99,1059.14,974.60,0.0,100.0
196,2021-05-18,Admin,9.99,1059.28,974.74,0.0,100.0
198,2021-05-23,Admin,9.99,1059.29,974.74,0.0,100.0
200,2021-05-25,Admin,9.99,1059.29,974.74,0.0,100.0
...,...,...,...,...,...,...,...
2646,2020-12-15,System,9.99,4947.81,83121.83,0.0,100.0
2766,2021-04-14,System,9.99,5028.24,83184.73,0.0,100.0
2800,2021-05-18,System,9.99,6833.58,85056.29,0.0,100.0
2802,2021-05-23,System,9.99,7057.54,85214.31,0.0,100.0


In [14]:
# Use the number of the day before 5 days from the missing date to impute the invalid capacity value
# e.g. use 2020-12-10 to impute 2020-12-15

missing_index = data[data["capacity (GiB)"] < 100].index
correct_index = missing_index-5

In [21]:
# The missing datas are: 
set(data["date"][missing_index])

{Timestamp('2020-12-15 00:00:00'),
 Timestamp('2021-04-14 00:00:00'),
 Timestamp('2021-05-18 00:00:00'),
 Timestamp('2021-05-23 00:00:00'),
 Timestamp('2021-05-25 00:00:00')}

In [22]:
data["capacity (GiB)"][missing_index] = data.loc[correct_index]["capacity (GiB)"]

<ipython-input-22-5dbcf4b0c923>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["capacity (GiB)"][missing_index] = data.loc[correct_index]["capacity (GiB)"]


In [23]:
# update the used percentage using the cleaned capacity
data["used %"] = round(data["used physical (GiB)"] / data["capacity (GiB)"] * 100,1)

## MinMax Scaler

In [24]:
data['Physical Normalised'] = data.groupby('volume name')['used physical (GiB)'].apply(lambda x: (x-x.min())/(x.max()-x.min()))

In [25]:
data['Logical Normalised'] = data.groupby('volume name')['used logical (GiB)'].apply(lambda x: (x-x.min())/(x.max()-x.min()))

## One Abnormal Data

Haven't figured it out..

In [26]:
data[data["capacity (GiB)"] < data["used physical (GiB)"]]

,date,volume name,capacity (GiB),used physical (GiB),used logical (GiB),free (GiB),used %,Physical Normalised,Logical Normalised
849,2021-05-23,DBkp,98303.96,112634.13,2101326.37,0.0,114.6,1.0,0.571499


---

## Save the preprocessed data

In [29]:
data = data.sort_values(["volume name", "date"], ascending = (True, True))
mypath = "/stornext/Home/data/allstaff/z/zhou.yo"
os.chdir(mypath)
data.to_csv("merge_data.csv",index = False)

In [30]:
data

,date,volume name,capacity (GiB),used physical (GiB),used logical (GiB),free (GiB),used %,Physical Normalised,Logical Normalised
0,2020-11-03,Admin,32767.98,1080.94,996.14,31687.05,3.3,0.708957,0.709283
1,2020-11-04,Admin,32767.98,1080.94,996.14,31687.05,3.3,0.708957,0.709283
2,2020-11-05,Admin,32767.98,1080.94,996.14,31687.04,3.3,0.708957,0.709283
3,2020-11-06,Admin,32767.98,1080.94,996.14,31687.04,3.3,0.708957,0.709283
4,2020-11-07,Admin,32767.98,1080.94,996.14,31687.04,3.3,0.708957,0.709283
...,...,...,...,...,...,...,...,...,...
2816,2021-06-06,System,32767.98,9943.52,87830.62,22824.47,30.3,0.859741,0.863340
2817,2021-06-07,System,32767.98,10135.55,87990.45,22632.43,30.9,0.892729,0.891697
2818,2021-06-08,System,32767.98,10327.59,88179.21,22440.39,31.5,0.925719,0.925188
2819,2021-06-09,System,32767.98,10567.96,88410.56,22200.02,32.3,0.967012,0.966235
